In [2]:
#default_exp models

In [3]:
#hide
from nbdev import *
%load_ext autoreload
%autoreload 2

# Models

> Uniserie models implementations.

In [4]:
#hide
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [117]:
#export
from itertools import count
from numbers import Number
from sys import float_info
from typing import Collection, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd
from numba import njit
from scipy.optimize import minimize
from statsmodels.regression.quantile_regression import QuantReg
from statsmodels.tsa.stattools import adfuller

from statsforecast.arima import auto_arima_f, forecast_arima

In [107]:
#exporti
@njit
def _ses_fcst_mse(x: np.ndarray, alpha: float) -> Tuple[float, float]:
    """Perform simple exponential smoothing on a series.

    This function returns the one step ahead prediction
    as well as the mean squared error of the fit.
    """
    smoothed = x[0]
    n = x.size
    mse = 0.

    for i in range(1, n):
        smoothed = (alpha * x[i - 1] + (1 - alpha) * smoothed).item()
        error = x[i] - smoothed
        mse += error * error

    mse /= n
    forecast = alpha * x[-1] + (1 - alpha) * smoothed
    return forecast, mse


def _ses_mse(alpha: float, x: np.ndarray) -> float:
    """Compute the mean squared error of a simple exponential smoothing fit."""
    _, mse = _ses_fcst_mse(x, alpha)
    return mse


@njit
def _ses_forecast(x: np.ndarray, alpha: float) -> float:
    """One step ahead forecast with simple exponential smoothing."""
    forecast, _ = _ses_fcst_mse(x, alpha)
    return forecast


@njit
def _demand(x: np.ndarray) -> np.ndarray:
    """Extract the positive elements of a vector."""
    return x[x > 0]


@njit
def _intervals(x: np.ndarray) -> np.ndarray:
    """Compute the intervals between non zero elements of a vector."""
    y = []

    ctr = 1
    for val in x:
        if val == 0:
            ctr += 1
        else:
            y.append(ctr)
            ctr = 1

    y = np.array(y)
    return y


@njit
def _probability(x: np.ndarray) -> np.ndarray:
    """Compute the element probabilities of being non zero."""
    return (x != 0).astype(np.int32)


def _optimized_ses_forecast(x: np.ndarray,
                            bounds: Sequence[Tuple[float, float]] = [(0.1, 0.3)]
                            ) -> float:
    """Searches for the optimal alpha and computes SES one step forecast."""
    alpha = minimize(
        fun=_ses_mse,
        x0=(0,),
        args=(x,),
        bounds=bounds,
        method='L-BFGS-B'
    ).x[0]
    forecast = _ses_forecast(x, alpha)
    return forecast


@njit
def _chunk_sums(array: np.ndarray, chunk_size: int) -> np.ndarray:
    """Splits an array into chunks and returns the sum of each chunk."""
    n = array.size
    n_chunks = n // chunk_size
    sums = np.empty(n_chunks)
    for i, start in enumerate(range(0, n, chunk_size)):
        sums[i] = array[start : start + chunk_size].sum()
    return sums

@njit
def embed(x: np.array, p: Union[Tuple, int]) -> np.array:
    """Embeds the time series x into a low-dimensional Euclidean space.
    
    Parameters
    ----------
    x: numpy array
        Time series.
    p: Iterable or int
        Embedding dimension.
        
    Notes
    -----
    [1] embed(x, p) = embed(x, [0, 1, ..., p - 1])
    
    References
    ----------
    [1] https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/embed
    """
    is_p_int = isinstance(p, int)

    if is_p_int and p == 0:
        raise Exception('Embedding dimension should not be 0')

    rolls = range(p) if is_p_int else p
    min_p = p - 1 if is_p_int else max(p)
    x_embed = np.full((len(x), len(rolls)), np.nan)
    for i, k in enumerate(rolls):
        x_embed[:, i] = np.roll(x, k)
    x_embed = x_embed[min_p:]

    return x_embed

In [108]:
embed(x, (0, 1, 2))

array([[2., 1., 0.],
       [3., 2., 1.],
       [4., 3., 2.],
       [5., 4., 3.],
       [6., 5., 4.],
       [7., 6., 5.],
       [8., 7., 6.],
       [9., 8., 7.]])

In [109]:
#hide
x = np.arange(10)
for emb in [embed(x, 3), embed(x, (0, 1, 2))]:
    np.testing.assert_equal(emb[:, 0], x[2:])
    np.testing.assert_equal(emb[:, 1], x[1:-1])
    np.testing.assert_equal(emb[:, 2], x[:-2])
    
def test_embed_equality(y, p):
    equivalent_list = tuple(range(p))
    np.testing.assert_equal(embed(y, p), embed(y, equivalent_list))
    
y = np.arange(100)
for i in range(1, 10): 
    test_embed_equality(y, i)
    
def test_p_zero():
    embed(y, 0)
    
test_fail(test_p_zero, contains='dimension should not be 0')

In [134]:
#export
def quantile_auto_regression(X, h, future_xreg, tau, ar_terms, 
                             add_constant=True, 
                             max_diffs=10):
    min_ar, max_ar = np.min(ar_terms), np.max(ar_terms)
    y = X[:, 0] if X.ndim == 2 else X
    last_y_train = y[-1]
    is_constant = np.var(y) == 0
    # Convert y to an stationary process
    differences = 0
    for _ in range(max_diffs):
        _, pval, *_ = adfuller(y)
        if pval < 0.05:
            break
        y = np.diff(y, 1)
        differences += 1
    design_mat = embed(y, (0, *ar_terms))
    y_train, X_train = design_mat[:, 0], design_mat[:, 1:]
    if add_constant:
        X_train = np.hstack([X_train, np.ones((len(X_train), 1))])
    if np.linalg.cond(X_train) > 1 / float_info.epsilon and not is_constant:
        raise Exception('X matrix is ill-conditioned '
                        'try reducing number of ar_terms '
                        'or setting add_constant=False.')
    model_ = QuantReg(y_train, X_train).fit(tau)
    y_hat = y_train
    len_train = y_train.shape[0]
    forecast_size = len_train + h
    while y_hat.size < forecast_size:
        y_hat_placeholder = np.zeros(min_ar)
        y_hat = np.concatenate([y_hat, y_hat_placeholder])
        y_test = embed(y_hat, ar_terms)[-min_ar:]
        if add_constant:
            y_test = np.hstack([y_test, np.ones((len(y_test), 1))])
        y_hat[-min_ar:] = model_.predict(y_test)
    y_hat = y_hat[len_train:forecast_size]
    if differences:
        for i in range(differences): y_hat = y_hat.cumsum()
        y_hat += last_y_train
    return y_hat

In [7]:
#export
@njit
def ses(X, h, future_xreg, alpha):
    y = X[:, 0] if X.ndim == 2 else X
    fcst, _ = _ses_fcst_mse(y, alpha)
    return np.full(h, fcst, np.float32)


def adida(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean()
    aggregation_level = round(mean_interval)
    lost_remainder_data = len(y) % aggregation_level
    y_cut = y[lost_remainder_data:]
    aggregation_sums = _chunk_sums(y_cut, aggregation_level)
    sums_forecast = _optimized_ses_forecast(aggregation_sums)
    forecast = sums_forecast / aggregation_level
    return np.repeat(forecast, h)


@njit
def historic_average(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    return np.repeat(y.mean(), h)


@njit
def croston_classic(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp = _ses_forecast(yd, 0.1)
    yip = _ses_forecast(yi, 0.1)
    return ydp / yip


@njit
def croston_sba(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    return 0.95 * croston_classic(y, h, future_xreg)


def croston_optimized(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp = _optimized_ses_forecast(yd)
    yip = _optimized_ses_forecast(yi)
    return ydp / yip


@njit
def seasonal_window_average(
    X: np.ndarray,
    h: int,
    future_xreg,
    season_length: int,
    window_size: int,
) -> np.ndarray:
    y = X[:, 0] if X.ndim == 2 else X
    min_samples = season_length * window_size
    if y.size < min_samples:
        return np.full(h, np.nan, np.float32)
    season_avgs = np.zeros(season_length, np.float32)
    for i, value in enumerate(y[-min_samples:]):
        season = i % season_length
        season_avgs[season] += value / window_size
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_avgs[i % season_length]
    return out


@njit
def seasonal_naive(X, h, future_xreg, season_length):
    return seasonal_window_average(X, h, future_xreg, season_length, 1)


def imapa(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean().item()
    max_aggregation_level = round(mean_interval)
    forecasts = np.empty(max_aggregation_level, np.float32)
    for aggregation_level in range(1, max_aggregation_level + 1):
        lost_remainder_data = len(y) % aggregation_level
        y_cut = y[lost_remainder_data:]
        aggregation_sums = _chunk_sums(y_cut, aggregation_level)
        forecast = _optimized_ses_forecast(aggregation_sums)
        forecasts[aggregation_level - 1] = (forecast / aggregation_level)
    forecast = forecasts.mean()
    return np.repeat(forecast, h)


@njit
def naive(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    return np.repeat(y[-1], h)


@njit
def random_walk_with_drift(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    slope = (y[-1] - y[0]) / (y.size - 1)
    return slope * (1 + np.arange(h)) + y[-1]


@njit
def window_average(X, h, future_xreg, window_size):
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < window_size:
        return np.full(h, np.nan, np.float32)
    wavg = y[-window_size:].mean()
    return np.repeat(wavg, h)


@njit
def seasonal_exponential_smoothing(X, h, future_xreg, season_length, alpha):
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < season_length:
        return np.full(h, np.nan, np.float32)
    season_vals = np.empty(season_length, np.float32)
    for i in range(season_length):
        season_vals[i] = _ses_forecast(y[i::season_length], alpha)
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_vals[i % season_length]
    return out


@njit
def tsb(X, h, future_xreg, alpha_d, alpha_p):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    yd = _demand(y)
    yp = _probability(y)
    ypf = _ses_forecast(yp, alpha_p)
    ydf = _ses_forecast(yd, alpha_d)
    forecast = np.float32(ypf * ydf)
    return np.repeat(forecast, h)

In [ ]:
#export
def auto_arima(X: np.ndarray, h: int, future_xreg=None, season_length: int = 1, 
               approximation: bool = False, level: Optional[Tuple[int]] = None) -> np.ndarray:
    y = X[:, 0] if X.ndim == 2 else X
    xreg = X[:, 1:] if (X.ndim == 2 and X.shape[1] > 1) else None
    mod = auto_arima_f(
        y, 
        xreg=xreg,
        period=season_length, 
        approximation=approximation,
        allowmean=False, allowdrift=False #not implemented yet
    )
    fcst = forecast_arima(mod, h, xreg=future_xreg, level=level)
    if level is None:
        return fcst['mean']
    return {
        'mean': fcst['mean'],
        **{f'lo-{l}': fcst['lower'][f'{l}%'] for l in reversed(level)},
        **{f'hi-{l}': fcst['upper'][f'{l}%'] for l in level},
    }    

In [87]:
from statsforecast.utils import AirPassengers as ap

In [ ]:
auto_arima(ap, 12, season_length=12)

External regressors

In [ ]:
drift = np.arange(1, ap.size + 1)
X = np.vstack([ap, np.log(drift), np.sqrt(drift)]).T

In [ ]:
newdrift = np.arange(ap.size + 1, ap.size + 7 + 1).reshape(-1, 1)
newxreg = np.concatenate([np.log(newdrift), np.sqrt(newdrift)], axis=1)

In [ ]:
auto_arima(X, 7, future_xreg=newxreg, season_length=12)

Confidence intervals

In [ ]:
pd.DataFrame(auto_arima(ap, 12, season_length=12, level=(80, 95)))